In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import pandas as pd
import numpy as np
from timeit import default_timer as timer
import scipy.io as sio
from scipy.special import comb

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.svm import SVC
from scipy.stats import expon
from sklearn.model_selection import RandomizedSearchCV
from sklearn.manifold import spectral_embedding
import math


import ensemble_learning as el

# Functions

In [ ]:
def onegramsTransform(data,dim1,dim2,minb=2,maxbh=30,cv = 5):
    #Removing missing values and store the result in the same variable
    data.dropna(inplace=True)
    #Grouping data by series
    group = data.groupby('Series')
    #Getting the different classes
    classe = group['classe'].apply(lambda x : x.iloc[0]).reset_index(drop=True)
    #Providing train/test indices to split data in train/test sets. Split dataset into cv consecutive folds, and shuffling the data before it
    kf = KFold(n_splits=cv,shuffle=True)
    mscore = 0
    #b1 from 2 to 29...by default
    for b1 in range(minb,maxbh):
        hist = group.apply(lambda x : np.ndarray.tolist(np.ndarray.flatten(
                    np.histogram(x.iloc[:,dim1].values,bins=b1,density=True)[0])))
        hist_tr = pd.DataFrame(hist.values.tolist())

        clf= KNeighborsClassifier(1)
        clf.fit(hist_tr, classe)

        scores = cross_val_score(clf, hist_tr, classe, cv=kf)

        if mscore < np.median(scores):
            
            mscore = np.median(scores)
            ubh1=b1
            
    #print("Bins: ", [ubh1], " \nMean scores : ", mscore)       
            
    mscore=0
    for b1 in range(minb,maxbh):
        hist = group.apply(lambda x : np.ndarray.tolist(np.ndarray.flatten(
                    np.histogram(x.iloc[:,dim2].values,bins=b1,density=True)[0])))
        hist_tr = pd.DataFrame(hist.values.tolist())

        clf=KNeighborsClassifier(1)
        clf.fit(hist_tr, classe)

        scores = cross_val_score(clf, hist_tr, classe, cv=kf)

        if mscore < np.median(scores):
            
            mscore = np.median(scores)
            ubh2=b1

    #print("Bins: ", [ubh2], " \nMean scores : ", mscore)

    return ubh1,ubh2

In [ ]:
def bigramsTransform(data,dim1,dim2,minb=2,maxb=11,cv = 5):
    data.dropna(inplace=True)
    group = data.groupby('Series')
    classe = group['classe'].apply(lambda x : x.iloc[0]).reset_index(drop=True)
    kf = KFold(n_splits=cv,shuffle=True)
    mscore = 0
    for b1 in range(minb,maxb):
        for b2 in range(minb,maxb):
            hist = group.apply(lambda x : np.ndarray.tolist(np.ndarray.flatten(
                        np.histogram2d(x.iloc[:,dim1].values,x.iloc[:,dim2].values,bins=[b1,b2],density=True)[0])))
            hist_tr = pd.DataFrame(hist.values.tolist())

            clf= KNeighborsClassifier(1)
            clf.fit(hist_tr, classe)

            scores = cross_val_score(clf, hist_tr, classe, cv=kf)

            if mscore < np.median(scores):
                
                mscore = np.median(scores)
                ub1=b1
                ub2=b2
    #print("Bins: ", [ub1,ub2], " \nMean scores : ", mscore) 
    return ub1,ub2

# Datasets

### Text

In [ ]:
dataset = [["ArabicDigits/ArabicDigits"," "],#0
           ["AUSLAN/AUSLAN"," ","\t"],#1
           ["CharacterTrajectories/CharacterTrajectories","\t"], #2          
           ["CMUsubject16/CMUsubject16","\t"],#3
           ["ECG/ECG"," "],#4
           ["Libras/Libras"," "],#5
           ["PenDigits/PenDigits","\t"],#6
           ['UWave/uWave'," "],#7
           ['RobotFailure/LP1'," "],#8
           ['RobotFailure/LP2'," "],#9
           ['RobotFailure/LP3'," "],#10
           ['RobotFailure/LP4'," "],#11
           ['RobotFailure/LP5'," "],#12
           ['Wafer/Wafer'," "],#13
           ["JapaneseVowels/JapaneseVowels", " "], #14
           ["ArticularyWordRecognition/ArticularyWordRecognition", " "],#15
           ["BasicMotions/BasicMotions", " "], #16
           ["Cricket/Cricket", " "], #17
           ["DuckDuckGeese/DuckDuckGeese", " "], #18
           ["EigenWorms/EigenWorms", " "], #19
           ["FingerMovements/FingerMovements", " "], #20
           ["HandMovementDirection/HandMovementDirection", " "], #21
           ["Heartbeat/Heartbeat", " "], #22
           ["LSST/LSST", " "], #23
           ["MotorImagery/MotorImagery", " "], #24
           ["NATOPS/NATOPS", " "], #25
           ["RacketSports/RacketSports", " "], #26
           ["SelfRegulationSCP1/SelfRegulationSCP1", " "], #27
           ["SelfRegulationSCP2/SelfRegulationsSCP2", " "] #28
              ]

In [ ]:
i=21

train = pd.read_table("MTS_Datasets/"+dataset[i][0]+"_TRAIN",sep=dataset[i][1],header=None)


# Renommage des 3 premières colonnes représentant resp. les numéros des STM,
# les indices de leurs obs et leur classe
train.rename(index=str, columns={0: "Series", 1: "index", 2:"classe"},inplace=True)
# Suppression de la colonne des index des obs des STM, infos inutiles pour les futures traitements
# (d'où l'utilisation de inplace=True) sinon le dataframe appelant la méthode drop serait inchangé et 
# une copie modifiée serait retournée en résultat
train.drop(['index'],axis=1,inplace=True) 
# Suppression et sauvegarde de la colonne des classes et celle des séries
cl = train.pop('classe') 
se = train.pop('Series')
# Insertion des colonnes précédemment supprimées à la fin du dataframe
train['classe'] = cl
train['Series'] = se
# Convertion du type des colonnes en string
train.columns = train.columns.astype(str)
print(train.head())


test = pd.read_table("MTS_Datasets/"+dataset[i][0]+"_TEST",sep=dataset[i][1],header=None)
test.rename(index=str, columns={0: "Series", 1: "index", 2:"classe"},inplace=True)
test.drop(['index'],axis=1,inplace=True)
cl = test.pop('classe')
se = test.pop('Series')
test['classe'] = cl
test['Series'] = se
test.columns = test.columns.astype(str)

            3          4          5           6           7           8  \
0  152.472131  12.392351  18.801367  119.067984    9.371185  176.878290   
1  151.149948  77.166998   7.669297   92.378960   28.596053  134.774572   
2  133.185425  75.533099  20.021506   50.618334   33.540720   77.921054   
3  105.026618  78.709435  -8.449302  -24.030543  -80.624318    8.100860   
4   96.332784  90.910153 -62.002411  -36.308074 -163.957234   14.436097   

            9          10          11          12  classe  Series  
0  187.217536  148.597142  192.406885  125.757108       1       1  
1  181.612639   -7.859846  123.777559  -38.720199       1       1  
2  143.941572  -75.591813   37.682789  -86.323079       1       1  
3   -8.680976 -132.255432   -1.438948  -37.793645       1       1  
4  -63.609558 -136.074317   84.123071   -3.151767       1       1  


### Matlab

In [ ]:
mat = sio.loadmat('MTS_Datasets/PEMS/PEMS.mat')
mdata = mat['mts']
mdtype = mdata.dtype  
ndata = {n: mdata[n][0, 0] for n in mdtype.names}
columns = [n for n, v in ndata.items()]

c='train'
train = pd.DataFrame()
for i in range(0,len(ndata[c][0])):
    series = pd.DataFrame(np.transpose(ndata[c][0][i]))
    series['classe']=ndata['trainlabels'][0][i]
    series['Series']=i
    train = pd.concat([train,series])
train.reset_index(drop=True,inplace=True)


c='test'
test = pd.DataFrame()
for i in range(0,len(ndata[c][0])):
    series = pd.DataFrame(np.transpose(ndata[c][0][i]))
    series['classe']=ndata['testlabels'][0][i]
    series['Series']=i
    test = pd.concat([test,series])
test.reset_index(drop=True,inplace=True)

In [ ]:
mat = sio.loadmat('MTS_Datasets/JapaneseVowels/JapaneseVowels.mat')
mdata = mat['mts']
mdtype = mdata.dtype  
ndata = {n: mdata[n][0, 0] for n in mdtype.names}
columns = [n for n, v in ndata.items()]

c='train'
train = pd.DataFrame()
for i in range(0,len(ndata[c][0])):
    series = pd.DataFrame(np.transpose(ndata[c][0][i]))
    series['classe']=ndata['trainlabels'][i][0]  # if Japanese Vowels
    #series['classe']=ndata['trainlabels'][0][i]
    series['Series']=i
    train = pd.concat([train,series])
train.reset_index(drop=True,inplace=True)


c='test'
test = pd.DataFrame()
for i in range(0,len(ndata[c][0])):
    series = pd.DataFrame(np.transpose(ndata[c][0][i]))
    series['classe']=ndata['testlabels'][i][0]     # if Japanese Vowels
    #series['classe']=ndata['testlabels'][0][i]
    series['Series']=i
    test = pd.concat([test,series])
test.reset_index(drop=True,inplace=True)

# Pre_treatments

In [ ]:
# Détermination du nbre d'instances ou observations composant (les dimensions des) les STM
nb_col = len(train.drop(['classe','Series'],axis=1).columns)
print(nb_col)
print(train.head())

10
            3          4          5           6           7           8  \
0  152.472131  12.392351  18.801367  119.067984    9.371185  176.878290   
1  151.149948  77.166998   7.669297   92.378960   28.596053  134.774572   
2  133.185425  75.533099  20.021506   50.618334   33.540720   77.921054   
3  105.026618  78.709435  -8.449302  -24.030543  -80.624318    8.100860   
4   96.332784  90.910153 -62.002411  -36.308074 -163.957234   14.436097   

            9          10          11          12  classe  Series  
0  187.217536  148.597142  192.406885  125.757108       1       1  
1  181.612639   -7.859846  123.777559  -38.720199       1       1  
2  143.941572  -75.591813   37.682789  -86.323079       1       1  
3   -8.680976 -132.255432   -1.438948  -37.793645       1       1  
4  -63.609558 -136.074317   84.123071   -3.151767       1       1  


## Normalisation

In [ ]:
train.iloc[:,0:nb_col] = train.groupby('Series').transform(lambda x: (x - x.min()) / (x.max()-x.min())).iloc[:,0:nb_col]
test.iloc[:,0:nb_col] = test.groupby('Series').transform(lambda x: (x - x.min()) / (x.max()-x.min())).iloc[:,0:nb_col]

In [ ]:
print(train)

              3         4         5         6         7         8         9  \
0      0.754349  0.666922  0.617529  0.854880  0.613323  0.833209  0.773135   
1      0.752683  0.748586  0.605668  0.817689  0.635159  0.778454  0.767421   
2      0.730046  0.746526  0.618829  0.759494  0.640775  0.704519  0.729021   
3      0.694564  0.750531  0.588494  0.655469  0.511106  0.613720  0.573445   
4      0.683609  0.765913  0.531435  0.638360  0.416456  0.621959  0.517454   
...         ...       ...       ...       ...       ...       ...       ...   
63995  0.503725  0.335637  0.457429  0.480824  0.564866  0.493387  0.672866   
63996  0.607420  0.376014  0.482064  0.518447  0.692324  0.708024  0.799360   
63997  0.627006  0.384698  0.505556  0.531257  0.693084  0.761330  0.807695   
63998  0.617961  0.385621  0.470202  0.562756  0.582686  0.738414  0.694918   
63999  0.596075  0.382868  0.396359  0.549316  0.501977  0.694901  0.609682   

             10        11        12  classe  Series

In [ ]:
mask  = train.isna().all()

#Drop columns where all elements are missing and keep the dataframe with valid entries in the same variable
train.dropna(axis=1,how='all',inplace=True)

train.interpolate(inplace=True)
train.dropna(inplace=True)

In [ ]:
test = test[test.columns[~mask]]
test.dropna(axis=1,how='all',inplace=True)
test.interpolate(inplace=True)
test.dropna(inplace=True)

In [ ]:
print(train.head(401))

            3         4         5         6         7         8         9  \
0    0.754349  0.666922  0.617529  0.854880  0.613323  0.833209  0.773135   
1    0.752683  0.748586  0.605668  0.817689  0.635159  0.778454  0.767421   
2    0.730046  0.746526  0.618829  0.759494  0.640775  0.704519  0.729021   
3    0.694564  0.750531  0.588494  0.655469  0.511106  0.613720  0.573445   
4    0.683609  0.765913  0.531435  0.638360  0.416456  0.621959  0.517454   
..        ...       ...       ...       ...       ...       ...       ...   
396  0.553296  0.748621  0.637124  0.918330  0.703182  0.773564  0.609546   
397  0.565759  0.739280  0.648544  0.925413  0.744639  0.738139  0.580484   
398  0.630152  0.732568  0.767051  0.927562  0.879331  0.712392  0.588253   
399  0.598498  0.737850  0.815125  0.917400  0.904102  0.697302  0.607643   
400  0.422042  0.441776  0.543537  0.368657  0.571177  0.269917  0.512898   

           10        11        12  classe  Series  
0    0.759200  0.798597

Saving the first serie into csv file to compute intrinsic dimensionality

In [ ]:
train.iloc[:400, 0:10].to_csv("handMovementDirection.csv", index=False , header=False,sep=" ")

# Manifold learning

In [ ]:
grouped_df = train.groupby("Series")
aaaaa = []
for key, item in grouped_df:
    #print(grouped_df.get_group(key).shape[0], "\n\n")
    aaaaa.append(grouped_df.get_group(key).shape[0])

aaaaa.sort()
print(aaaaa)

[400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400]


In [ ]:
!pip install tslearn
from tslearn.metrics import dtw

In [ ]:
def reduce(data, nbcol, dim=3, repetition=False):
  # data is a dataframe representing a serie of the dataset
  size = data.shape[0]
    
  # If the number or data points if less or equal to the reduced dimension wished, drop the series
  if dim >= size:
    data.drop(data.index, inplace=True)
    return data
  #if size > 25:
  #  repetition = True
    

  A = np.zeros((size, size))
  np.fill_diagonal(A, 1) 
  R = np.zeros((size, size)) # Repetition neighbourhood matrix
  

  # Handling the extreme data points
  A[0,1] = math.exp(-np.linalg.norm(data.iloc[0,:nbcol] - data.iloc[1,:nbcol]))
  A[1,0] = A[0,1]
  end = size - 1 # Last index of A
  bend = end - 1
  A[end,bend] = math.exp(-np.linalg.norm(data.iloc[end,:nbcol] - data.iloc[bend,:nbcol]))
  A[bend,end] = A[end,bend]

  if repetition == True: # We compute simultaneously the adjacent temporal neighbourhood matrix and the repetition temporal neighbourhood matrix

    endRepetition = end-10 # Last index of the data points while computing the repetition neighbourhoods
    fragmentNumbers = endRepetition - 9 # Number of fragments
    M = np.zeros((fragmentNumbers, fragmentNumbers)) # Similarity matrix of the fragments
    fragments = list() # List containing the fragments. Each element of the list contains the coordinates (extracted dataframe) of the data points in the fragment
    M_mod = np.zeros((fragmentNumbers, fragmentNumbers)) # Similarity matrix M after windowing

    # Handling the intermediate data points for adjacent neighbours
    for i in range(1,end):
      A[i,i-1] =  math.exp(-np.linalg.norm(data.iloc[i,:nbcol] - data.iloc[i-1,:nbcol]))
      A[i-1,i] = A[i,i-1]
      A[i,i+1] = math.exp(-np.linalg.norm(data.iloc[i,:nbcol] - data.iloc[i+1,:nbcol]))
      A[i+1,i] = A[i,i+1]
      if i >= 10 and i <= endRepetition:
        fragments.append(data.iloc[i-10:i+11, :nbcol])

    # Computing the M matrix
    for i in range(0,fragmentNumbers):
      for j in range(0,i):
        M[i,j] = dtw(fragments[i], fragments[j])
        M[j,i] = M[i,j]

    # Performing temporal windowing of M
    for i in range(0,fragmentNumbers):
      for j in range(i+1, fragmentNumbers):
        for b in range(0, 20):
          if i-b > 0 and j-b >0:  #  ET LES AUTRES ?
            M_mod[i,j] = M_mod[i,j] + M[i-b,j-b]
        M_mod[i,j] = M_mod[i,j] / 20
        M_mod[j,i] = M_mod[i,j]
        
      

    
    # Searching for similar fragments, extracting the similar fragments and building the matrix R
    b = 0.75
    Bool = (M_mod < M_mod.mean(axis=1) - b * M_mod.std(axis=1))
    for i in range(0, fragmentNumbers):
      for j in range(i+1, fragmentNumbers):
        pointI = i+10 # Center of the fragment
        pointJ = j+10
        if Bool[i,j]:
          if A[pointI,pointJ] != 0 and A[pointI,pointJ] != 1 :
            R[pointI,pointJ] = A[pointI,pointJ]
          elif A[pointI,pointJ] == 0:
            R[pointI,pointJ] = math.exp(-np.linalg.norm(data.iloc[pointI,:nbcol] - data.iloc[pointJ,:nbcol]))
          R[pointJ,pointI] = R[pointI,pointJ]



  else: # We compute only the adjacent temporal neighbourhood matrix
    

    # Handling the intermediate data points for adjacent neighbours
    for i in range(1,end):
      A[i,i-1] =  math.exp(-np.linalg.norm(data.iloc[i,:nbcol] - data.iloc[i-1,:nbcol]))
      A[i-1,i] = A[i,i-1]
      A[i,i+1] = math.exp(-np.linalg.norm(data.iloc[i,:nbcol] - data.iloc[i+1,:nbcol]))
      A[i+1,i] = A[i,i+1]
  
  #to_drop = ["7", "8", "9", "10", "11", "12", "13", "14", "15"]
  data.drop(data.iloc[:, dim:nbcol], axis=1, inplace=True)
  #data.drop(columns=to_drop, inplace=True)
  data.iloc[:,:dim] = spectral_embedding(A+R,dim)
  return data



def dimensionReduction(data, nbcol, dim=3):
  #data = data.dropna()
  group = data.groupby("Series")
  reducedData = group.apply(reduce, nbcol, dim)
  return reducedData

train2 = train
test2 = test

dim = 3

train_r = dimensionReduction(train, nb_col, dim)
print(train_r)

              3         4         5  classe  Series
0      0.056261  0.055056 -0.054789       1       1
1      0.056259  0.055049 -0.054774       1       1
2      0.056255  0.055031 -0.054734       1       1
3      0.056246  0.054997 -0.054658       1       1
4      0.056235  0.054954 -0.054561       1       1
...         ...       ...       ...     ...     ...
63995  0.055687 -0.055613 -0.055896       4     160
63996  0.055702 -0.055677 -0.056039       4     160
63997  0.055710 -0.055710 -0.056113       4     160
63998  0.055716 -0.055731 -0.056160       4     160
63999  0.055717 -0.055738 -0.056175       4     160

[64000 rows x 5 columns]


In [ ]:
# Quatre voisins

def reduce(data, nbcol, dim=3, repetition=True):
  # data is a dataframe representing a serie of the dataset
  size = data.shape[0]
    
  # If the number or data points if less or equal to the reduced dimension wished, drop the series
  if dim >= size:
    data.drop(data.index, inplace=True)
    return data
  #if size > 25:
  #  repetition = True
    

  A = np.zeros((size, size))
  np.fill_diagonal(A, 1) 
  R = np.zeros((size, size)) # Repetition neighbourhood matrix
  

  # Handling the extreme data points
  A[0,1] = math.exp(-np.linalg.norm(data.iloc[0,:nbcol] - data.iloc[1,:nbcol]))
  A[1,0] = A[0,1]
  end = size - 1 # Last index of A
  bend = end - 1
  A[end,bend] = math.exp(-np.linalg.norm(data.iloc[end,:nbcol] - data.iloc[bend,:nbcol]))
  A[bend,end] = A[end,bend]

  if repetition == True: # We compute simultaneously the adjacent temporal neighbourhood matrix and the repetition temporal neighbourhood matrix

    endRepetition = end-10 # Last index of the data points while computing the repetition neighbourhoods
    fragmentNumbers = endRepetition - 9 # Number of fragments
    M = np.zeros((fragmentNumbers, fragmentNumbers)) # Similarity matrix of the fragments
    fragments = list() # List containing the fragments. Each element of the list contains the coordinates (extracted dataframe) of the data points in the fragment
    M_mod = np.zeros((fragmentNumbers, fragmentNumbers)) # Similarity matrix M after windowing

    # Handling the intermediate data points for adjacent neighbours
    for i in range(1,end):
      A[i,i-1] =  math.exp(-np.linalg.norm(data.iloc[i,:nbcol] - data.iloc[i-1,:nbcol]))
      A[i-1,i] = A[i,i-1]
      A[i,i+1] = math.exp(-np.linalg.norm(data.iloc[i,:nbcol] - data.iloc[i+1,:nbcol]))
      A[i+1,i] = A[i,i+1]
      if i >= 10 and i <= endRepetition:
        fragments.append(data.iloc[i-10:i+11, :nbcol])

    # Computing the M matrix
    for i in range(0,fragmentNumbers):
      for j in range(0,i):
        M[i,j] = dtw(fragments[i], fragments[j])
        M[j,i] = M[i,j]

    # Performing temporal windowing of M
    for i in range(0,fragmentNumbers):
      for j in range(i+1, fragmentNumbers):
        for b in range(0, 20):
          if i-b > 0 and j-b >0:  #  ET LES AUTRES ?
            M_mod[i,j] = M_mod[i,j] + M[i-b,j-b]
        M_mod[i,j] = M_mod[i,j] / 20
        M_mod[j,i] = M_mod[i,j]
        
      

    
    # Searching for similar fragments, extracting the similar fragments and building the matrix R
    b = 0.75
    Bool = (M_mod < M_mod.mean(axis=1) - b * M_mod.std(axis=1))
    for i in range(0, fragmentNumbers):
      for j in range(i+1, fragmentNumbers):
        pointI = i+10 # Center of the fragment
        pointJ = j+10
        if Bool[i,j]:
          if A[pointI,pointJ] != 0 and A[pointI,pointJ] != 1 :
            R[pointI,pointJ] = A[pointI,pointJ]
          elif A[pointI,pointJ] == 0:
            R[pointI,pointJ] = math.exp(-np.linalg.norm(data.iloc[pointI,:nbcol] - data.iloc[pointJ,:nbcol]))
          R[pointJ,pointI] = R[pointI,pointJ]



  else: # We compute only the adjacent temporal neighbourhood matrix
    

    # Handling the intermediate data points for adjacent neighbours
    for i in range(2,end-1):
      A[i,i-1] =  math.exp(-np.linalg.norm(data.iloc[i,:nbcol] - data.iloc[i-1,:nbcol]))
      A[i-1,i] = A[i,i-1]
      A[i,i-2] =  math.exp(-np.linalg.norm(data.iloc[i,:nbcol] - data.iloc[i-2,:nbcol]))
      A[i-2,i] = A[i,i-2]
      A[i,i+1] = math.exp(-np.linalg.norm(data.iloc[i,:nbcol] - data.iloc[i+1,:nbcol]))
      A[i+1,i] = A[i,i+1]
      A[i,i+2] =  math.exp(-np.linalg.norm(data.iloc[i,:nbcol] - data.iloc[i+2,:nbcol]))
      A[i+2,i] = A[i,i+2]
  
  #to_drop = ["7", "8", "9", "10", "11", "12", "13", "14", "15"]
  data.drop(data.iloc[:, dim:nbcol], axis=1, inplace=True)
  #data.drop(columns=to_drop, inplace=True)
  data.iloc[:,:dim] = spectral_embedding(A+R,dim)
  return data



def dimensionReduction(data, nbcol, dim=3):
  #data = data.dropna()
  group = data.groupby("Series")
  reducedData = group.apply(reduce, nbcol, dim)
  return reducedData

train2 = train
test2 = test

dim = 2

train_r = dimensionReduction(train, nb_col, dim)
print(train_r)

In [ ]:
test_r = dimensionReduction(test, nb_col, dim)
train = train2
test = test2
print(test_r)

## Corr coeff

In [ ]:
start = timer()


nb_col = len(train_r.drop(['classe','Series'],axis=1).columns)
nb_col

# Calcul de la dérivée et de la somme cumulée
deriv = train_r.groupby('Series').transform(lambda x: x.diff()).iloc[:,0:nb_col]

cumsum = train_r.groupby('Series').transform(lambda x: x.cumsum()).iloc[:,0:nb_col]

data = pd.concat([train_r.iloc[:,0:nb_col],deriv,cumsum],axis=1)
data.columns = np.linspace(0,3*nb_col,3*nb_col,dtype=int)

train_r = pd.concat([data,train_r[['classe','Series']]], axis=1)
train_r.dropna(inplace=True)

#print(train_r)

deriv = test_r.groupby('Series').transform(lambda x: x.diff()).iloc[:,0:nb_col]

cumsum = test_r.groupby('Series').transform(lambda x: x.cumsum()).iloc[:,0:nb_col]

data = pd.concat([test_r.iloc[:,0:nb_col],deriv,cumsum],axis=1)
data.columns = np.linspace(0,3*nb_col,3*nb_col,dtype=int)

test_r = pd.concat([test_r.iloc[:,0:nb_col],deriv,cumsum,test_r[['classe','Series']]],
          axis=1)
test_r.dropna(inplace=True)

#print(test_r)


####### M-histogramme #########

# Paramètres
max_comb = comb(nb_col,2)*3*2


print("Nombre de M-histogrammes : ",max_comb)

eval_int = 0
# Apprentissage vue
for app in range(0,4):
    print("*****************************")
    print("Apprentissage: ",app)
    print("*****************************")
    train_nngrams = list() #Liste des M-Histogrammes de l'ensemble d'apprentissage pour chaque vue
    test_nngrams = list() #Liste des M-Histogrammes de l'ensemble de test
    learners = list()
    combin=list()           #Liste des combinaisons dim1+dim2+TypeHistogramme
    
    # Apprentissage M-histogramme
    i=0
    while i <nb_features and i<max_comb :
        #if i%5==0:
            #print("*****************************")
            #print("TOUR : ",i)
            #print("*****************************")
        # Choosing : Dimensional features ||  Simple, Deriv or Cumsum ||  Bigrams or 1grams 
        dim1rand = np.random.randint(0,nb_col)
        dim2rand = np.random.randint(0,nb_col)
        while dim2rand == dim1rand :
            dim2rand = np.random.randint(0,nb_col)


        transform = np.random.randint(0,3)
        dim1 = dim1rand+transform*nb_col
        dim2 = dim2rand+transform*nb_col

        nngrams = np.random.randint(0,2)    

        #print("Dim1 : ",dim1,"   Dim2 : ",dim2,"    NNGrams: ",nngrams)

        if [dim1,dim2,nngrams] in combin or [dim2,dim1,nngrams] in combin : 
            continue
        else :
            combin.append([dim1,dim2,nngrams]) 
            i+=1

        if nngrams == 0:

            ub1,ub2 = bigramsTransform(train_r.copy(),dim1,dim2)

            train_group = train_r.groupby('Series')        
            htr = train_group.apply(lambda x : np.ndarray.tolist(np.ndarray.flatten(
                        np.histogram2d(x.iloc[:,dim1].values,x.iloc[:,dim2].values,bins=[ub1,ub2],density=True)[0])))
            htr = pd.DataFrame(htr.values.tolist())
            train_nngrams.append(htr)


            test_group = test_r.groupby('Series')
            htt = test_group.apply(lambda x : np.ndarray.tolist(np.ndarray.flatten(
                        np.histogram2d(x.iloc[:,dim1].values,x.iloc[:,dim2].values,bins=[ub1,ub2],density=True)[0])))
            htt = pd.DataFrame(htt.values.tolist())
            test_nngrams.append(htt)

            learners.append(('clf',KNeighborsClassifier(1)))


        else :

            ub1h,ub2h = onegramsTransform(train_r.copy(),dim1,dim2) 

            train_group = train_r.groupby('Series')
            h1htr = train_group.apply(lambda x : np.ndarray.tolist(np.ndarray.flatten(
                        np.histogram(x.iloc[:,dim1].values,bins=ub1h,density=True)[0])))
            h1htr = pd.DataFrame(h1htr.values.tolist())
            train_nngrams.append(h1htr)

            h2htr = train_group.apply(lambda x : np.ndarray.tolist(np.ndarray.flatten(
                        np.histogram(x.iloc[:,dim2].values,bins=ub2h,density=True)[0])))
            h2htr = pd.DataFrame(h2htr.values.tolist())
            train_nngrams.append(h2htr)


            test_group = test_r.groupby('Series')
            h1htt = test_group.apply(lambda x : np.ndarray.tolist(np.ndarray.flatten(
                        np.histogram(x.iloc[:,dim1].values,bins=ub1h,density=True)[0])))
            h1htt = pd.DataFrame(h1htt.values.tolist())
            test_nngrams.append(h1htt)

            h2htt = test_group.apply(lambda x : np.ndarray.tolist(np.ndarray.flatten(
                        np.histogram(x.iloc[:,dim2].values,bins=ub2h,density=True)[0])))
            h2htt = pd.DataFrame(h2htt.values.tolist())
            test_nngrams.append(h2htt)

            learners.append(('clf',KNeighborsClassifier(1)))
            learners.append(('clf',KNeighborsClassifier(1)))

    # Score en validation
    cl = train_group['classe'].apply(lambda x : x.iloc[0]).reset_index(drop=True) #Liste des classes pour chaque série, en ordre
    se = np.arange(len(train_nngrams[0]))
    #train_index : index des séries de l'ensemble d'apprentissage, y_train: classes des séries de l'ensemble d'apprentissage, indexées par les index des séries
    train_index, test_index, y_train, y_test = train_test_split(se, cl,stratify=cl,test_size=0.2) 

    x_train = list()
    x_test = list()
    for j in range(len(train_nngrams)): #Chaque item de x_train contiendra pour une vue la liste des histogrammes des séries d'entraînement
        x_train.append(train_nngrams[j].iloc[train_index,:].reset_index(drop=True))  
    for j in range(len(train_nngrams)):
        x_test.append(train_nngrams[j].iloc[test_index,:].reset_index(drop=True))

    fitted_estimators, label_encoder = el.fit_multiple_estimators(learners, x_train, y_train.reset_index(drop=True))
    y_pred = el.predict_from_multiple_estimator(fitted_estimators, label_encoder, x_test)
    score_val = np.round(accuracy_score(y_pred, y_test.reset_index(drop=True)),4)
    print("SCORE Training VUE N°",app," : ",score_val )
    
    # Score en test
    train_group = train_r.groupby('Series')
    train_classe = train_group['classe'].apply(lambda x : x.iloc[0]).reset_index(drop=True)

    test_group = test_r.groupby('Series')
    test_classe = test_group['classe'].apply(lambda x : x.iloc[0]).reset_index(drop=True)

    fitted_estimators, label_encoder = el.fit_multiple_estimators(learners, train_nngrams, train_classe)
    y_pred = el.predict_from_multiple_estimator(fitted_estimators, label_encoder, test_nngrams)
    
    score_test = np.round(accuracy_score(y_pred, test_classe),4)
    print("SCORE TEST: ", score_test)
  
    
    if eval_int < score_val :
        eval_int=score_val
        eval_fi=score_test
        
print("SCORE FINAL: ", eval_fi)

end = timer()
print(end - start)